In [104]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
%autoreload

import pandas as pd
import utilities as utils
import numpy as np

from catboost import CatBoostRegressor
from catboost import Pool
from sklearn.model_selection import train_test_split

In [106]:
data_list = utils.read_data()
Data = utils.DataProcessor(data_list)
df = Data.create_train_data() # NaNs are filled with 0
df.sort_index(inplace=True)
df = df.loc['2019-09-04 08:00:00':]

c:\Users\elias\Documents\TDT4173-ML\utilities.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\elias\Documents\TDT4173-ML\utilities.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = pd.concat([X_train, y_train], axis=1)


In [107]:
X = df.drop(['pv_measurement'], axis=1)
y = df['pv_measurement']

In [108]:
categorical_features_indices = np.where(X.dtypes != float)[0]

In [109]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.80, random_state=11568)

In [110]:
model = CatBoostRegressor(
            loss_function='MAE',
            depth=6,
            learning_rate=0.1,
            iterations=1000,
            random_seed=18,
)

In [111]:
train_pool = Pool(X_train, y_train)
validation_pool = Pool(X_validation, y_validation)


In [112]:
model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False
)

In [113]:
X_test = Data.create_test_data().drop(['building'], axis=1)
# X_test
# X_test = X_test[utils.get_features(3)]
X_test.fillna(0, inplace=True)

test_pool = Pool(X_test)

In [117]:
prediction = model.predict(X_test)
# prediction = np.exp(prediction)-1


In [115]:
prediction.dtype

dtype('float64')

In [118]:
preds_df = pd.DataFrame(prediction, columns=['predict'])
preds_df[preds_df < 0] = 0

results = pd.concat([X_test.reset_index(), preds_df], axis=1)
results['date_forecast'] = pd.to_datetime(results['date_forecast'])
results.set_index('date_forecast', inplace=True)
results.rename(columns={'predict': 'pv_prediction'}, inplace=True)

utils.save_to_csv(results, 'catboost_1')